In [1]:
import pandas as pd
import json
 
# Opening JSON file
with open('data/data.json') as json_file:
    data = json.load(json_file)

In [2]:
# Creating an empty Dataframe with column names only
dfObj = pd.DataFrame(columns=['label', 'text', 'rank'])
print("Empty Dataframe ", dfObj, sep='\n')

Empty Dataframe 
Empty DataFrame
Columns: [label, text, rank]
Index: []


In [5]:
def filter_max_rank(data, max_rank):
    '''
    only keep top ranks up to max_rank
    '''
    dfObj = pd.DataFrame(columns=['label', 'text', 'rank'])
    for key in data:
        jump = False
        for i in range(len(data[key])):
            if i > max_rank: #ignore i > max_rank
                jump = True
                break;
            tmp = {}
            tmp['label'] = key
            tmp['text'] = data[key][i]
            tmp['rank'] = i+1
            dfObj = dfObj.append(tmp, ignore_index=True)
        if jump:
            jump = False
            continue;
    return dfObj

In [6]:
max_rank = 100
dfObj = filter_max_rank(data, max_rank)
dfObj

,label,text,rank
0,Pipers Great Berwick Longhorn Beef Crisps 150g,Pipers Great Berwick Longhorn Beef Crisps 150g,1
1,Pipers Great Berwick Longhorn Beef Crisps 150g,Pipers Crisps - Great Berwick Longhorn Beef (1...,2
2,Pipers Great Berwick Longhorn Beef Crisps 150g,Pipers Crisps Great Berwick Longhorn Beef Flav...,3
3,Pipers Great Berwick Longhorn Beef Crisps 150g,Pipers Great Berwick Longhorn Beef Crisps,4
4,Pipers Great Berwick Longhorn Beef Crisps 150g,Pipers Longhorn Beef Crisps 40g - 24 Pack,5
...,...,...,...
3731,Pipers Kirkby Malham Chorizo Sharing Crisps 150g,Pipers Crisps Anglesey Sea Salt 3 Case Sizes A...,56
3732,Pipers Kirkby Malham Chorizo Sharing Crisps 150g,Sustainable Dairy Box | Pipers Farm,57
3733,Pipers Kirkby Malham Chorizo Sharing Crisps 150g,Pipers Lye Cross Cheddar & Onion Crisps 150g,58
3734,Pipers Kirkby Malham Chorizo Sharing Crisps 150g,SMALL Meat Box | Delivery & Collection | Parso...,59


In [1]:
import numpy as np
import spacy

# Need to load the large model to get the vectors
nlp = spacy.load('en_core_web_lg')

In [51]:
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(text).vector for text in dfObj.text])
    
doc_vectors.shape

(3736, 300)

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(doc_vectors, dfObj.label,
                                                    test_size=0.1, random_state=1)

In [53]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Set dual=False to speed up training, and it's not needed
svc = LinearSVC(random_state=1, dual=False, max_iter=10000)
svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svc.fit(X_train, y_train)
print(f"Accuracy: {svc.score(X_test, y_test) * 100:.3f}%", )

Accuracy: 48.663%


In [54]:
max_accuracy = 0
for max_rank in range(3, 31, 1):
    dfObj = filter_max_rank(data, max_rank)
    dfObj
    with nlp.disable_pipes():
        doc_vectors = np.array([nlp(text).vector for text in dfObj.text])
    X_train, X_test, y_train, y_test = train_test_split(doc_vectors, dfObj.label,
                                                        test_size=0.1, random_state=1)
    #svc = LinearSVC(random_state=1, dual=False, max_iter=10000)
    svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    svc.fit(X_train, y_train)
    acc = svc.score(X_test, y_test)
    if acc > max_accuracy:
        best_rank = max_rank
        max_accuracy = acc
    print(max_rank, f"Accuracy: {acc * 100:.3f}%", )
print(best_rank)

3 Accuracy: 40.000%
4 Accuracy: 51.613%
5 Accuracy: 65.789%
6 Accuracy: 72.727%
7 Accuracy: 58.000%
8 Accuracy: 64.286%
9 Accuracy: 54.839%
10 Accuracy: 53.623%
11 Accuracy: 52.000%
12 Accuracy: 64.198%
13 Accuracy: 66.667%
14 Accuracy: 63.441%
15 Accuracy: 59.000%
16 Accuracy: 65.094%
17 Accuracy: 66.071%
18 Accuracy: 59.322%
19 Accuracy: 61.290%
20 Accuracy: 56.489%
21 Accuracy: 58.394%
22 Accuracy: 57.343%
23 Accuracy: 48.993%
24 Accuracy: 56.129%
25 Accuracy: 58.025%
26 Accuracy: 57.738%
27 Accuracy: 55.172%
28 Accuracy: 53.333%
29 Accuracy: 53.763%
30 Accuracy: 55.959%
6


In [55]:
max_rank = 10
dfObj = filter_max_rank(data, max_rank)
dfObj
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(text).vector for text in dfObj.text])
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, dfObj.label,
                                                    test_size=0.1, random_state=1)
#svc = LinearSVC(random_state=1, dual=False, max_iter=10000)
svc = make_pipeline(StandardScaler(), SVC(gamma='auto',probability=True))
svc.fit(X_train, y_train)
print(max_rank, f"Accuracy: {svc.score(X_test, y_test) * 100:.3f}%", )

10 Accuracy: 53.623%


In [56]:
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(text).vector for text in ["Hula Hoops Original Potato Ring Crisps 6x24g"]])
print(doc_vectors)

[[ 8.71253200e-04  3.05572629e-01  5.32430001e-02 -6.83925003e-02
  -2.39938740e-02  3.10222469e-02  4.56349831e-03 -1.21091880e-01
   6.49937540e-02  5.21313727e-01 -1.41460493e-01  7.20899925e-02
   1.70787990e-01 -5.27282469e-02  4.05593723e-01  2.24412456e-02
   1.24242879e-01  7.11930275e-01  1.79250017e-02 -1.78741843e-01
   2.94376016e-01 -7.54189938e-02 -4.68849987e-02  6.08843863e-02
   1.33625254e-01 -1.23123378e-01 -3.03660631e-01 -3.53889838e-02
  -6.66411296e-02 -1.36976004e-01 -2.26286113e-01  6.61347508e-02
  -1.23761490e-01 -1.14333242e-01  1.66716874e-01 -3.63573767e-02
  -1.82500109e-04  6.39811233e-02  1.27934992e-01  6.24454916e-02
  -7.93290958e-02  3.87161255e-01 -2.82199979e-02  1.05793104e-01
   8.86519849e-02  1.76485002e-01 -9.35624540e-03  8.57737362e-02
   6.64749742e-03  2.45998397e-01 -2.53140070e-02  6.15366250e-02
   1.61667496e-01 -8.97574946e-02  3.90399955e-02 -2.31400996e-01
   4.75913621e-02  1.59345493e-01  6.27917498e-02  5.38476296e-02
  -1.70961

In [57]:
y = svc.predict_proba(doc_vectors)
print(np.max(y))
print(np.min(y))
print(np.max(y)/np.mean(y))

0.06372798173811278
0.0044502282446666494
3.951134867762992


In [58]:
y = svc.predict_proba(doc_vectors)
print(np.max(y))
print(np.min(y))
print(np.max(y)/np.min(y)>15)

0.06372798173811278
0.0044502282446666494
False


In [59]:
import pickle
with open("data/html", "rb") as fp:   # Unpickling
   b = pickle.load(fp)
for u in b:
    print(u)

Log in / Register
Search a list of items
Favourites
Nectar
Offers
Discover
Recipes
Delivery Pass
Occasions
Mother's Day
Sainsbury's Sea Salt & Cider Vinegar Crisps, Taste the Difference 150g
(31)
Sainsbury's Cheddar & Spring Onion Crisps, Taste the Difference 150g
(36)
Pringles Prawn Cocktail 200g
(5)
Kettle Chips Lightly Salted Sharing Crisps 150g
(14)
Only £1.25: Save 75p
Sensations Thai Sweet Chilli Sharing Crisps 150g
(9)
Only £1.25: Save 75p
Kettle Chips Sea Salt & Balsamic Vinegar of Modena Sharing Crisps 150g
(10)
Only £1.25: Save 75p
Kettle Chips Sea Salt & Crushed Black Peppercorns Sharing Crisps 150g
(9)
Only £1.25: Save 75p
Pringles Original Crisps 200g
(9)
Kettle Chips Mature Cheddar & Red Onion Sharing Crisps 150g
(28)
Only £1.25: Save 75p
Kettle Chips Sweet Chilli & Sour Cream Sharing Crisps 150g
(13)
Only £1.25: Save 75p
Doritos Cool Original Sharing Tortilla Chips Crisps 180g
(5)
Walkers Classic Variety Multipack Crisps 22x25g
Jacob's Cracker Crisps Sour Cream & Chive 1

In [60]:
def use_model(html):
    names = []
    for unit in html:
        with nlp.disable_pipes():
            doc_vector = np.array([nlp(unit).vector])
        y = svc.predict_proba(doc_vector)
        if np.max(y)/np.min(y)>8:
            names.append(unit)
    return list(set(names))

names = use_model(b)
print(len(names))
for u in names:
    print(u)

125
Sainsbury's Ridge Classic Variety Ridge Cut Assorted Crisps 6x27g
1
Sensations Roast Chicken & Thyme Sharing Crisps 150g
Pipers Great Berwick Longhorn Beef Crisps 150g
Doritos Lightly Salted Sharing Tortilla Chips Crisps 270g
Pipers Lye Cross Cheddar & Onion Sharing Crisps 150g
Recipes
(36)
(6)
Cookie Policy
(13)
3
(3)
Pringles Original Crisps 200g
(28)
(65)
Doritos Tangy Cheese Sharing Tortilla Chips Crisps 180g
Tyrrells Cheddar Cheese & Chive Sharing Crisps 150g
Pipers Burrow Hill Cider Vinegar & Sea Salt Sharing Crisps 150g
Pringles Prawn Cocktail 200g
Doritos Flamin' Hot Tangy Cheese Sharing Tortilla Chips Crisps 180g
Accessibility
(10)
Only £1.25: Save £1.25
Pringles Sizzl'N Extra Hot Cheese & Chilli Crisps 180g
(1)
Sainsbury's Sea Salt & Cider Vinegar Crisps, Taste the Difference 150g
Cookie policy
Kettle Chips Sea Salt & Balsamic Vinegar of Modena Sharing Crisps 150g
Kettle Chips Sea Salt & Crushed Black Peppercorns Sharing Crisps 150g
Jacob's Twiglets Crisps 150g
(24)
Walke

In [61]:
names = []
for unit in b:
    with nlp.disable_pipes():
        doc_vector = np.array([nlp(unit).vector])
    y = svc.predict_proba(doc_vector)
    if np.max(y)/np.min(y)>15:
        names.append(unit)
names = list(set(names))
print(len(names))
for u in names:
    print(u)

123
Sainsbury's Ridge Classic Variety Ridge Cut Assorted Crisps 6x27g
1
Sensations Roast Chicken & Thyme Sharing Crisps 150g
Pipers Great Berwick Longhorn Beef Crisps 150g
Doritos Lightly Salted Sharing Tortilla Chips Crisps 270g
Pipers Lye Cross Cheddar & Onion Sharing Crisps 150g
Recipes
(36)
(6)
Cookie Policy
(13)
3
(3)
Pringles Original Crisps 200g
(28)
(65)
Doritos Tangy Cheese Sharing Tortilla Chips Crisps 180g
Tyrrells Cheddar Cheese & Chive Sharing Crisps 150g
Pipers Burrow Hill Cider Vinegar & Sea Salt Sharing Crisps 150g
Pringles Prawn Cocktail 200g
Doritos Flamin' Hot Tangy Cheese Sharing Tortilla Chips Crisps 180g
Accessibility
(10)
Only £1.25: Save £1.25
Pringles Sizzl'N Extra Hot Cheese & Chilli Crisps 180g
(1)
Sainsbury's Sea Salt & Cider Vinegar Crisps, Taste the Difference 150g
Cookie policy
Kettle Chips Sea Salt & Balsamic Vinegar of Modena Sharing Crisps 150g
Kettle Chips Sea Salt & Crushed Black Peppercorns Sharing Crisps 150g
Jacob's Twiglets Crisps 150g
(24)
Walke

In [62]:
import pickle
with open("data/html_eval", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
names = use_model(b)

for ratio in range(0,30,2):
    names = []
    for unit in b:
        with nlp.disable_pipes():
            doc_vector = np.array([nlp(unit).vector])
        y = svc.predict_proba(doc_vector)
        if np.max(y)/np.min(y)>ratio:
            names.append(unit)
    names = list(set(names))

    ##eval
    with open("data/truth", "rb") as fp:   # Unpickling
       truth = pickle.load(fp)

    TP = len(set.intersection(set(truth), set(names)))
    FP = len(set(names)) - TP
    FN = len(set(truth)) - TP
    print((TP,FP,FN))
    F1 = TP/(TP + 0.5*(FP+FN))
    print((ratio,F1))

(55, 98, 314)
(0, 0.210727969348659)
(55, 98, 314)
(2, 0.210727969348659)
(55, 98, 314)
(4, 0.210727969348659)
(55, 98, 314)
(6, 0.210727969348659)
(55, 98, 314)
(8, 0.210727969348659)
(55, 98, 314)
(10, 0.210727969348659)
(55, 98, 314)
(12, 0.210727969348659)
(55, 95, 314)
(14, 0.2119460500963391)
(52, 88, 317)
(16, 0.2043222003929273)
(52, 68, 317)
(18, 0.21267893660531698)
(39, 55, 330)
(20, 0.16846652267818574)
(23, 24, 346)
(22, 0.11057692307692307)
(16, 3, 353)
(24, 0.08247422680412371)
(0, 1, 369)
(26, 0.0)
(0, 0, 369)
(28, 0.0)


In [63]:
with open("data/html_eval2", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
print(len(b))
for bi in b:
    print(bi)

174
Log in / Register
Search a list of items
Favourites
Nectar
Offers
Discover
Recipes
Delivery Pass
Occasions
Mother's Day
Sainsbury's Sea Salt & Cider Vinegar Crisps, Taste the Difference 150g
(31)
Sainsbury's Cheddar & Spring Onion Crisps, Taste the Difference 150g
(36)
Pringles Prawn Cocktail 200g
(5)
Kettle Chips Lightly Salted Sharing Crisps 150g
(14)
Only £1.25: Save 75p
Sensations Thai Sweet Chilli Sharing Crisps 150g
(9)
Only £1.25: Save 75p
Kettle Chips Sea Salt & Balsamic Vinegar of Modena Sharing Crisps 150g
(10)
Only £1.25: Save 75p
Kettle Chips Sea Salt & Crushed Black Peppercorns Sharing Crisps 150g
(9)
Only £1.25: Save 75p
Pringles Original Crisps 200g
(9)
Kettle Chips Mature Cheddar & Red Onion Sharing Crisps 150g
(28)
Only £1.25: Save 75p
Kettle Chips Sweet Chilli & Sour Cream Sharing Crisps 150g
(13)
Only £1.25: Save 75p
Doritos Cool Original Sharing Tortilla Chips Crisps 180g
(5)
Walkers Classic Variety Multipack Crisps 22x25g
Jacob's Cracker Crisps Sour Cream & Chi

In [64]:
with open("data/truth2", "rb") as fp:   # Unpickling
   truth = pickle.load(fp)
print(len(truth))
for ti in truth:
    print(ti)

62
Pipers Great Berwick Longhorn Beef Crisps 150g
Tyrrells Sweet Chilli & Red Pepper Sharing Crisps 150g
Sainsbury's Sea Salt & Cider Vinegar Crisps, Taste the Difference 150g
Pringles Prawn Cocktail 200g
Sensations Balsamic Vinegar & Caramelised Onion Sharing Crisps 150g
Walkers Ready Salted Crisps 45g
Walkers Classic Variety Multipack Crisps 22x25g
Doritos Flamin' Hot Tangy Cheese Sharing Tortilla Chips Crisps 180g
Sainsbury's Gourmet Sea Salt Crisps, Taste the Difference 150g
Walkers Cheese & Onion Crisps 45g
Popchips Barbeque Sharing Crisps 85g
Jacob's Cracker Crisps Sour Cream & Chive 150g
Tyrrells Lentil Sharing Crisps Sour Cream & Onion 80g
Tyrrells Cider Vinegar & Salt Sharing Crisps 150g
Sainsbury's Ready Salted Crisps 12x25g
Sainsbury's Vegetable Crisps, Taste the Difference 100g
Popchips Sea Salt & Vinegar Sharing Crisps 85g
Doritos Chilli Heatwave Sharing Tortilla Chips Crisps 180g
Walkers Roast Chicken Crisps 45g
Pringles Sizzl'N Kickin' Sour Cream Crisps 180g
Kettle Chips

In [69]:
import pickle
with open("data/html_eval2", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
#names = use_model(b)

for ratio in range(0,50,2):
    names = []
    for unit in b:
        with nlp.disable_pipes():
            doc_vector = np.array([nlp(unit).vector])
        y = svc.predict_proba(doc_vector)
        if np.max(y)/np.min(y)>ratio:
            names.append(unit)
    names = list(set(names))

    ##eval
    with open("data/truth2", "rb") as fp:   # Unpickling
       truth = pickle.load(fp)

    TP = len(set.intersection(set(truth), set(names)))
    FP = len(set(names)) - TP
    FN = len(set(truth)) - TP
    print((TP,FP,FN))
    F1 = TP/(TP + 0.5*(FP+FN))
    print((ratio,F1))

(62, 63, 0)
(0, 0.6631016042780749)
(62, 63, 0)
(2, 0.6631016042780749)
(62, 63, 0)
(4, 0.6631016042780749)
(62, 63, 0)
(6, 0.6631016042780749)
(62, 63, 0)
(8, 0.6631016042780749)
(62, 63, 0)
(10, 0.6631016042780749)
(62, 63, 0)
(12, 0.6631016042780749)
(62, 63, 0)
(14, 0.6631016042780749)
(62, 60, 0)
(16, 0.6739130434782609)
(62, 59, 0)
(18, 0.6775956284153005)
(61, 47, 1)
(20, 0.7176470588235294)
(61, 23, 1)
(22, 0.8356164383561644)
(61, 16, 1)
(24, 0.8776978417266187)
(60, 0, 2)
(26, 0.9836065573770492)
(60, 0, 2)
(28, 0.9836065573770492)
(60, 0, 2)
(30, 0.9836065573770492)
(59, 0, 3)
(32, 0.9752066115702479)
(59, 0, 3)
(34, 0.9752066115702479)
(59, 0, 3)
(36, 0.9752066115702479)
(59, 0, 3)
(38, 0.9752066115702479)
(59, 0, 3)
(40, 0.9752066115702479)
(59, 0, 3)
(42, 0.9752066115702479)
(58, 0, 4)
(44, 0.9666666666666667)
(55, 0, 7)
(46, 0.9401709401709402)
(55, 0, 7)
(48, 0.9401709401709402)


In [71]:
import pickle
with open("data/html_eval3", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
#names = use_model(b)

for ratio in range(0,50,2):
    names = []
    for unit in b:
        with nlp.disable_pipes():
            doc_vector = np.array([nlp(unit).vector])
        y = svc.predict_proba(doc_vector)
        if np.max(y)/np.min(y)>ratio:
            names.append(unit)
    names = list(set(names))

    ##eval
    with open("data/truth3", "rb") as fp:   # Unpickling
       truth = pickle.load(fp)

    TP = len(set.intersection(set(truth), set(names)))
    FP = len(set(names)) - TP
    FN = len(set(truth)) - TP
    print((TP,FP,FN))
    F1 = TP/(TP + 0.5*(FP+FN))
    print((ratio,F1))

(62, 72, 0)
(0, 0.6326530612244898)
(62, 72, 0)
(2, 0.6326530612244898)
(62, 72, 0)
(4, 0.6326530612244898)
(62, 72, 0)
(6, 0.6326530612244898)
(62, 72, 0)
(8, 0.6326530612244898)
(62, 72, 0)
(10, 0.6326530612244898)
(62, 72, 0)
(12, 0.6326530612244898)
(57, 72, 5)
(14, 0.5968586387434555)
(52, 69, 10)
(16, 0.5683060109289617)
(42, 67, 20)
(18, 0.49122807017543857)
(33, 51, 29)
(20, 0.4520547945205479)
(23, 22, 39)
(22, 0.42990654205607476)
(15, 16, 47)
(24, 0.3225806451612903)
(10, 0, 52)
(26, 0.2777777777777778)
(8, 0, 54)
(28, 0.22857142857142856)
(4, 0, 58)
(30, 0.12121212121212122)
(2, 0, 60)
(32, 0.0625)
(1, 0, 61)
(34, 0.031746031746031744)
(1, 0, 61)
(36, 0.031746031746031744)
(1, 0, 61)
(38, 0.031746031746031744)
(1, 0, 61)
(40, 0.031746031746031744)
(1, 0, 61)
(42, 0.031746031746031744)
(1, 0, 61)
(44, 0.031746031746031744)
(1, 0, 61)
(46, 0.031746031746031744)
(0, 0, 62)
(48, 0.0)


In [66]:
##eval
import pickle
with open("data/truth", "rb") as fp:   # Unpickling
   truth = pickle.load(fp)

TP = len(set.intersection(set(truth), set(names)))
FP = len(set(names)) - TP
FN = len(set(truth)) - TP
#print((TP,FP,FN))
F1 = TP/(TP + 0.5*(FP+FN))
print(F1)

0.0


In [3]:
def cosine_similarity(a, b):
    return a.dot(b)/np.sqrt(a.dot(a) * b.dot(b))

In [7]:
a = nlp("Trump").vector
b = nlp("Donald John Trump").vector
cosine_similarity(a, b)

0.7242044

In [68]:
a = nlp("Walkers Quavers Cheese Multipack Snacks 6x16g").vector
b = nlp("Walkers Quavers Cheese Snacks 20 X 16g").vector
cosine_similarity(a, b)

0.83876854

In [5]:
a = nlp("Walkers Quavers Cheese Multipack Snacks 6x16g").vector
b = nlp("Potato Hula Original Rings 34G (32 x 34G) Hoops").vector
cosine_similarity(a, b)

0.4377603

In [44]:
crisps = []
df2 = pd.DataFrame(columns=['label', 'score'])
b = nlp("Hula Hoops Original Potato Rings 34G (32 x 34G)").vector
for key in data:
    tmp = {}
    b = nlp(key).vector
    tmp['label'] = key
    tmp['score'] = cosine_similarity(a, b)
    df2 = df2.append(tmp, ignore_index=True)
df2

,label,score
0,Walkers Quavers Cheese Multipack Snacks 6x16g,1.000000
1,Jacob's Mini Cheddars Original x6 25g,0.467382
2,Hula Hoops Original Potato Ring Crisps 6x24g,0.623494
3,Walkers Salt & Vinegar Multipack Crisps 6x25g,0.771717
4,Walkers Prawn Cocktail Multipack Crisps 6x25g,0.804654
...,...,...
61,Eat Real Veggie Straws Kale Tomato Spinach 5x20g,0.668505
62,Harvest Snaps Thai Sweet Chilli Crispy Lentil ...,0.639202
63,Harvest Snaps Sour Cream & Chive Crunchy Lenti...,0.660516
64,Jacob's Mini Cheddars Chunchlets Rich & Tangy ...,0.613349


In [46]:
df2.sort_values(by=['score'], ascending=False)

,label,score
0,Walkers Quavers Cheese Multipack Snacks 6x16g,1.000000
10,Walkers Quavers Cheese Multipack Snacks 12x16g,0.941428
13,Walkers Baked Cheese & Onion Mulitpack Snacks ...,0.853342
39,Walkers Quavers Prawn Cocktail Multipack Snack...,0.852199
35,Walkers Cheese & Onion Crisps 12x25g,0.839790
...,...,...
52,Hippeas Chickpea Puffs Sweet & Smokin 5pk,0.511239
47,McCoy's Fire Pit Flame Roasted Peri Peri Multi...,0.506583
59,Hippeas Salt & Vinegar Vibes Mulltipack,0.496580
1,Jacob's Mini Cheddars Original x6 25g,0.467382
